# Estimaciones en una encuesta


In [2]:
# ---------- # Importación de bibliotecas # ------------------ #
import os
import sys
import pandas as pd

# ---------- # Importación de funciones # ------------------ #
from modulos.func_analisis import *
from modulos.func_transformacion import *

Dentro del script `func_analisis` se encuentran las funciones `proporciones`, `proporciones_des`, `prevalencias`, `prevalencias_des` y `tabulados_prevalencias`. Todas estas funciones se pueden utilizar para generar estimaciones a partir de los datos de cualquier encuesta con un muestreo probabilístico estratificado. 

A grandes razgos las funciones requieren como argumentos el nombre de las varibles donde se almacena la información del ponderador o factor de expansión, la unidad primaria de muestreo, el estrato, las preguntas que se desean analizar y el dataframe donde están dichas variables. 

Para usarlas, sólo es necesario definir correctamente sus argumentos. Los pasos que se recomienda seguir son:
1. Identificar las variables de interés en la base de datos asociada a la encuesta.
2. Homologar el tipo de datos de las variables de interés.
3. Estructurar un dataframe con las variables de interés.
4. Generar la estimación a partir de las respuestas a las preguntas seleccionadas.

A continuación se ejemplifica cómo se pueden generar estimaciones de alguna pregunta de interés con los datos de la Encuesta Nacional de Consumo de Drogas, Alcohol y Tabaco, ENCODAT 2016-2017.

In [3]:
# ------------------ # Carga de las bases de datos # ------------------ #
h=pd.read_csv(os.path.join('datos','originales','ENCODAT_2016_2017_Hogar','ENCODAT_2016_2017_Hogar.csv'), sep=';', low_memory=False)
bb=pd.read_csv(os.path.join('datos','originales','ENCODAT_2016_2017_Individual','ENCODAT_2016_2017_Individual.csv'), sep=';', low_memory=False)

## 1. Identificación de variables
En los argumentos de las funciones para hacer estimaciones, es necesario indicar el nombre de las variables que almacenan el factor de expansión, la unidad primaria de muestreo, el estrato y las preguntas de interés. Por ello, el primer paso es identificar si se tienen dichas variables y cuál es su nombre o clave.

En el caso de la ENCODAT:
- el factor de expansión está dentro de las variables `ponde_ss` y `ponde_hh`
- la unidad primaria de muestreo es la variable `code_upm`
- el estrato está en la variable `estrato`

Las preguntas de interés dependerán de la información que se desee estimar. 

Para este ejemplo, se eligieron dos preguntas:
_____
`ds8` **¿Actualmente estudia?**
1. No, nunca ha asistido a la escuela
2. No, pero si fue a la escuela
3. Sí
____
 
`ds9` **¿Cuál fue el último grado que ha completado o completó en la escuela?**
1. Primaria incompleta (1 a 5 años)
2. Primaria completa (6 años)
3. Secundaria incompleta (1 a 2 años)
4. Secundaria completa o equivalente (3 años)
5. Bachillerato incompleto (1 a 2 años)
6. Bachillerato completo o equivalente (aprox. 3 años)
7. Estudios Universitarios incompletos (1 a 3 años)
8. Estudios  Universitarios completos (4 a 5 años)
9. Estudios de Posgrado (2 a 4 años)
99) No contesta

In [4]:
# ---------- # Selección de preguntas # ------------------ #

#escribir aqui los códigos de las preguntas de interés
clv_preg = ['ds8','ds9']

## 2. Homologación de datos

Las variables que requieren las funciones de `func_analisis` deben ser datos de tipo numérico, por ello para empezar a hacer estimaciones es necesario hacer una pequeña exploración a las bases de datos y en caso de ser necesario cambiar los datos, codificarlos o corregirlos.

Siguiendo con el ejemplo de la ENCODAT, en la siguientes celdas se hace una pequeña exploración a las bases de datos y se homologan los datos a formato numérico.

### 2.1 Preparativos de las variables asociadas a la muestra

In [5]:
h[['estrato','code_upm','ponde_hh']].head()

,estrato,code_upm,ponde_hh
0,3,01_12,"170,475477457288"
1,3,01_12,"170,475477457288"
2,3,01_12,"170,475477457288"
3,3,01_12,"170,475477457288"
4,3,01_12,"170,475477457288"


In [6]:
h[['estrato','code_upm','ponde_hh']].dtypes

estrato      int64
code_upm    object
ponde_hh    object
dtype: object

Con las dos celdas anteriores se puede ver que la variable estrato ya tiene formato numérico, así que no vamos a hacerle ningún procesamiento. Sin embargo, las variables `code_upm` y `ponde_hh` sí necesitan homologarse a formato numérico, además la variable `ponde_hh` tiene más información que sólo el ponderador.

En la siguiente celda se hacen las adecuaciones correspondientes para solucionar los problemas anteriores:

In [7]:
# Homologación de codigos de las unidades primarias de muestreo para que sean sólo numéricos
h.insert(0,'upm',h['code_upm'].str.replace('_','').astype(int))

# Ahora bien, en el caso muy específico de esta base de datos,
#el factor de expansión son los primeros dígitos separados por la coma en la variable 'ponde_hh'
#así que se genera una nueva variable sólo con el ponderador en formato numérico
h.insert(0,'factor_exp',h['ponde_hh'].str.partition(',')[0].astype(int))

De igual forma, en la base 'bb', la variable "ponde_ss" tiene más información que sólo el ponderador, así que se corrige:

In [8]:
bb['ponde_ss'].head()

0    263,004588042305
1    779,543148763914
2    263,004588042305
3     281,00343231265
4    421,505148468975
Name: ponde_ss, dtype: object

In [9]:
# Se genera una nueva variable sólo con el ponderador en formato numérico
bb.insert(0,'factor_exp',bb['ponde_ss'].str.partition(',')[0].astype(int))

### 2.2 Preparativos de las variables asociadas a las preguntas
En el caso de la ENCODAT las respuestas a preguntas se codifican mediante claves, sin embargo no siempre están homologadas para que efectivamente sean sólo datos numéricos. En la siguiente celda podemos ver como una de las preguntas seleccionadas (`ds9`) tiene datos de distintos tipos, no sólo numéricos.

In [10]:
# ---------- # Revisión del tipo de respuestas de las preguntas seleccionadas # ------------------ #
bb[clv_preg].dtypes

ds8     int64
ds9    object
dtype: object

In [11]:
bb.ds9.unique()

array(['4', '6', '5', '1', '3', '2', '7', '9', '8', ' ', '99'],
      dtype=object)

En una inspección más detallada (en la celda anterior) podemos ver que hay dos problemas con el tipo de datos almacenados en la variable `ds9`:
- el primero es que hay espacios vacíos
- el segundo es que todas las claves son cadenas de caracteres.

Para solucionar ambos problemas se hace lo siguiente:

In [12]:
# ---------- # Homologación de los datos de las preguntas seleccionadas # ------------------ #
#pregunta con problemas
clave_pregunta = 'ds9'

#para solucionar el primer problema es necesario asignar una clave numérica a los datos vacíos
#en este caso se elige asignar el 0 para los espacios vacíos
bb[clave_pregunta]=bb[clave_pregunta].replace(' ',0)

#para solucionar el segundo problema se cambia el tipo de dato, de cadena de caracteres a numérico
bb[clave_pregunta]=bb[clave_pregunta].astype(int)

NOTA: la homologación y estructuración de una base de datos es de las partes más complejas del análisis de datos porque los datos pueden tener infinidad de variaciones o problemas, cada uno con su solución particular.

Sin embargo, una forma general de abordar esto es inspeccionar las respuestas y homologar la forma en la que se escriben dichas respuestas. 

## 3. Estructuración del dataframe

La información de la ENCODAT está distribuida en dos bases de datos (nombradas en este cuaderno como `bb` y `h`), por ello en este caso la estructuración del dataframe requiere unir dichas bases. 

Ahora bien, cada base puede necesitar sus adecuaciones particulares para estructurar el Dataframe que requieren las funciones del script`func_analisis`.



In [13]:
# ---------- # Creación de dataframe para funciones # ------------------ #

# Creación de variable 'id_hogar' para unir la base "bb" con la base "h"
bb.insert(0,'id_hogar',bb['id_pers'].str[:20])

# Unión de las bases en un solo dataframe mediante el id_hogar
b=bb.merge(h[['id_hogar','estrato','upm','code_upm','est_var']],on='id_hogar',how='left')

## 4. Estimación

### 4.1 ¿Con qué proporción se mencionó cada respuesta de la pregunta seleccionada?
Para saber con que proporción se mencionó cada respuesta de las preguntas seleccionadas se puede utilizar la función `proporciones`. Para utilizarla, basta llamar a la función y dar los siguientes argumentos:
- `b` dataframe con la base de datos.
- `clave_preg` cadena de caracteres con el nombre de la variable en la que están las respuestas de la pregunta para la que se desea estimar las proporciones.
- `alp` valor entre 0 y 1 para definir el nivel de significancia del intervalo (el valor predeterminado es 0.95).
- `ponderador` cadena de caracteres con el nombre de la variable donde está el factor de expansión de cada registro en la base de datos (el valor predeterminado es "factor_exp").
- `estrato` cadena de caracteres con el nombre de la variable donde están la información de los estratos de la encuesta (el valor predeterminado es "estrato").
- `upm` cadena de caracteres con el nombre de la variable donde está la información de las unidades primarias de la encuesta (el valor predeterminado es "upm").

Al ejecutar la celda, la función genera estimaciones puntuales, su respectivo intervalo de confianza, el coeficiente de variación y el error estándar:

In [14]:
proporciones(b = b, 
             clave_preg = 'ds9', 
             alp = 0.95, 
             ponderador = 'factor_exp', 
             estrato= 'estrato', 
             upm = 'upm')

,clave_respuesta,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
0,0,2.235656,1.998388,2.472924,1905531,0.121057,5.414847
1,1,7.543581,7.150328,7.936835,6429670,0.200643,2.659790
2,2,15.188253,14.563453,15.813053,12945503,0.318781,2.098869
3,3,10.088452,9.668178,10.508726,8598756,0.214429,2.125493
4,4,25.132433,24.390835,25.874031,21421291,0.378373,1.505518
5,5,8.865000,8.431773,9.298226,7555963,0.221038,2.493378
6,6,16.721276,15.973866,17.468685,14252154,0.381339,2.280559
7,7,5.092753,4.575115,5.610390,4340739,0.264106,5.185910
8,8,8.088889,7.508010,8.669769,6894456,0.296373,3.663948
9,9,0.884170,0.742965,1.025374,753610,0.072045,8.148282


La columna `estimación` del dataframe que genera la función `proporciones` describe con que proporción la población (estimada por la encuesta) eligió las respuestas.

Por ejemplo, en la ENCODAT se estima que el 15.18% de la población tiene la primaria completa (respuesta que corresponde a la clave 2)

### 4.2 ¿Con qué proporción se mencionó cada respuesta de la pregunta seleccionada? 
### Desagregado por otra variable

Si se desea saber con qué proporción se mencionó cada respuesta en subconjuntos definidos por otra pregunta, se puede utilizar la función `proporciones_des`.

La función utiliza los mismos argumentos que `proporciones` e incluye uno más: 
`var_des` lista con el(los) nombre(s) de la(s) variable(s) con respecto a la cual se quieren crear los subconjuntos.

Para mostrar concretamente lo que hace la función, aplicaremos la función `proporciones_des` para la pregunta `ds9` (nivel de escolaridad) y desagregaremos las respuestas de acuerdo con la pregunta `ds8` (actualmente estudia), es decir, podremos saber la proporción de las personas que estudian actualmente con base en su escolaridad:

In [15]:
estudiantes_por_escolaridad=proporciones_des(b = b,
                 clave_preg = 'ds9',
                 var_des = ['ds8'],
                 alp = 0.95,
                 ponderador = 'factor_exp',
                 estrato= 'estrato',
                 upm = 'upm')

estudiantes_por_escolaridad

,ds8,estimacion,clave_respuesta,ic_inf,ic_sup,poblacion,error_std,cv
20,1,100.000000,0,100.000000,100.000000,1905531,0.000000,0.000000
9,2,0.189053,99,0.125219,0.252888,121116,0.032569,17.227454
8,2,0.847453,9,0.697067,0.997839,542916,0.076729,9.054086
7,2,9.649621,8,8.947750,10.351492,6181975,0.358104,3.711067
6,2,3.029559,7,2.680650,3.378469,1940870,0.178018,5.876052
0,2,9.071511,1,8.579950,9.563072,5811612,0.250801,2.764709
4,2,6.528307,5,6.091566,6.965049,4182323,0.222831,3.413312
3,2,29.471576,4,28.597480,30.345672,18880798,0.445976,1.513239
2,2,6.347032,3,5.953384,6.740680,4066190,0.200844,3.164382
1,2,16.656890,2,15.909295,17.404485,10671142,0.381433,2.289942


Puede ser un poco complicado recordar a qué corresponde cada clave de respuesta, por ello, renombraremos las claves del anterior dataframe, para que sea más clara la estructura que regresa la función `proporciones_des`:

In [16]:
estudiantes_por_escolaridad.replace({'ds8':
                                    {1:'No, nunca ha asistido a la escuela',
                                     2:'No, pero si fue a la escuela',
                                     3:'Sí, actualmente estudia'},
                                   'clave_respuesta':
                                    {1:'Primaria incompleta', 
                                    2:'Primaria completa',
                                    3:'Secundaria incompleta',
                                    4:'Secundaria completa',
                                    5:'Bachillerato incompleto',
                                    6:'Bachillerato completo',
                                    7:'Estudios Universitarios incompletos',
                                    8:'Estudios  Universitarios completos',
                                    9:'Estudios de Posgrado',
                                    99:'No contesta',
                                    0:'Nunca ha asistido a la escuela',}}, inplace=True)

estudiantes_por_escolaridad.sort_values('ds8')

,ds8,estimacion,clave_respuesta,ic_inf,ic_sup,poblacion,error_std,cv
20,"No, nunca ha asistido a la escuela",100.000000,Nunca ha asistido a la escuela,100.000000,100.000000,1905531,0.000000,0.000000
1,"No, pero si fue a la escuela",16.656890,Primaria completa,15.909295,17.404485,10671142,0.381433,2.289942
2,"No, pero si fue a la escuela",6.347032,Secundaria incompleta,5.953384,6.740680,4066190,0.200844,3.164382
3,"No, pero si fue a la escuela",29.471576,Secundaria completa,28.597480,30.345672,18880798,0.445976,1.513239
4,"No, pero si fue a la escuela",6.528307,Bachillerato incompleto,6.091566,6.965049,4182323,0.222831,3.413312
5,"No, pero si fue a la escuela",18.208996,Bachillerato completo,17.380667,19.037325,11665490,0.422625,2.320966
6,"No, pero si fue a la escuela",3.029559,Estudios Universitarios incompletos,2.680650,3.378469,1940870,0.178018,5.876052
7,"No, pero si fue a la escuela",9.649621,Estudios Universitarios completos,8.947750,10.351492,6181975,0.358104,3.711067
8,"No, pero si fue a la escuela",0.847453,Estudios de Posgrado,0.697067,0.997839,542916,0.076729,9.054086
9,"No, pero si fue a la escuela",0.189053,No contesta,0.125219,0.252888,121116,0.032569,17.227454


Entonces, a partir de las estimaciones anteriores podemos decir que del total de personas que tienen estudios universitarios completos, el 9.64% actualmente no estudia y el 3.69% sigue estudiando.

### 4.3 ¿Con qué proporción se mencionaron ciertas respuesta de la pregunta seleccionada? 

En ocasiones se desea calcular el porcentaje de selección de cierto conjunto de respuestas, para estos casos se puede utilizar la función `prevalencias`. 

Los argumentos de la función son los siguientes:
- `b` dataframe con la base de datos.
- `lista_vars` lista con los nombres de las variables para las que se desea calcular prevalencia.
- `dicc`diccionario con la clasificación de las claves de respuesta.
- `alp` valor entre 0 y 1 para definir el nivel de significancia del intervalo (el valor predeterminado es 0.95).
- `ponderador` cadena de caracteres con el nombre de la variable donde está el factor de expansión de cada registro en la base de datos (el valor predeterminado es "factor_exp").
- `estrato` cadena de caracteres con el nombre de la variable donde están la información de los estratos de la encuesta (el valor predeterminado es "estrato").
- `upm` cadena de caracteres con el nombre de la variable donde está la información de las unidades primarias de la encuesta (el valor predeterminado es "upm").

Como se puede notar los argumentos son casi los mismos que las funciones `proporciones` y `proporciones_des`. El único argumento nuevo es `dicc`, que tiene como objetivo recategorizar las respuestas a las preguntas de la lista de variables 'lista_vars' de la siguiente manera: 
- 1 si el evento ocurrió
- 0 si el evento no ocurrió

Para que sea más claro, creemos un ejemplo. Supóngase que se desea obtener la proporción de las personas que actualmente no estudian a partir de la pregunta `ds8`. 

Existen dos posibles respuestas que indican que alguien no estudia: `1`: "No, nunca ha asistido a la escuela" y `2` "No, pero si fue a la escuela". 

El argumento dicc sirve para indicar qué respuestas se desean considerar para calcular la proporción de un evento. En este caso el *evento* es que alguien no estudie, por lo tanto:
- las respuestas con claves `1` y `2`se renombran como `1`porque indican que el evento ocurre.
- las respuestas con clave `3` se renombran como `0` porque indican que el evento no ocurre.

Es decir, para este caso específico el argumento `dicc` es:
{1:1, 2:1, 3:0}.

Veámos que produce la función una vez que son claros todos los argumentos que requiere:

In [17]:
prevalencias(b = b, 
             lista_vars = ['ds8'], 
             dicc={1:1, 2:1, 3:0},
             alp = 0.95, 
             ponderador = 'factor_exp', 
             estrato= 'estrato', 
             upm = 'upm')

,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
1,77.398962,76.697332,78.100593,65969963,0.357981,0.462514


El resultado ahora es mucho más concreto, sólo se desea saber el porcentaje de la población que se estima que no estudia, y este es 77.39%. 

Sólo para comprobar, saquemos la proporción de cada respuesta de la pregunta `ds8`:

In [18]:
proporciones(b = b, 
             clave_preg = 'ds8', 
             alp = 0.95, 
             ponderador = 'factor_exp', 
             estrato= 'estrato', 
             upm = 'upm')

,clave_respuesta,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
0,1,2.235656,1.998388,2.472924,1905531,0.121057,5.414847
1,2,75.163307,74.447193,75.879420,64064432,0.365371,0.486103
2,3,22.601038,21.899407,23.302668,19263690,0.357981,1.583915


Y bueno, efectivamente, la suma de las estimaciones para las claves `1` y `2` corresponden con la estimación hecha a partir de la función `prevalencias`.

### 4.4 ¿Con qué proporción se mencionaron ciertas respuesta de la pregunta seleccionada? 
### Desagregado por otra variable
Al igual que la función `proporciones` tiene la función `proporciones_des` para obtener estimaciones desagregadas por otra variable, la función `prevalencias` tiene la función más específica `prevalencias_des` para generar estimaciones desagregadas por otra variable.

Al igual que `proporciones_des`, `prevalencias_des` tiene el argumento `var_des` para indicar la lista de variables con respecto a las cuales se van a desagregar los resultados.

Siguiendo con el ejemplo anterior, obtengamos la proporción de personas que no estudia desagregadas por escolaridad:

In [25]:
prevalencias_des(b = b,
                 lista_vars = ['ds8'],
                 dicc={1:1, 2:1, 3:0},
                 var_des = ['ds9'],
                 alp = 0.95,
                 ponderador = 'factor_exp',
                 estrato= 'estrato',
                 upm = 'upm').replace({'ds9':{1:'Primaria incompleta', 
                                    2:'Primaria completa',
                                    3:'Secundaria incompleta',
                                    4:'Secundaria completa',
                                    5:'Bachillerato incompleto',
                                    6:'Bachillerato completo',
                                    7:'Estudios Universitarios incompletos',
                                    8:'Estudios  Universitarios completos',
                                    9:'Estudios de Posgrado',
                                    99:'No contesta',
                                    0:'Nunca ha asistido a la escuela',}})

,ds9,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
9,Nunca ha asistido a la escuela,100.000000,100.000000,100.000000,1905531,0.000000,0.000000
3,Primaria incompleta,90.387407,88.842200,91.932614,5811612,0.788385,0.872229
5,Primaria completa,82.431266,81.057882,83.804650,10671142,0.700719,0.850065
4,Secundaria incompleta,47.288119,45.279154,49.297085,4066190,1.025001,2.167566
0,Secundaria completa,88.140337,87.301344,88.979330,18880798,0.428066,0.485664
2,Bachillerato incompleto,55.351290,52.807981,57.894599,4182323,1.297630,2.344354
1,Bachillerato completo,81.850715,79.939909,83.761522,11665490,0.974919,1.191094
6,Estudios Universitarios incompletos,44.712893,39.912790,49.512997,1940870,2.449077,5.477340
8,Estudios Universitarios completos,89.665885,87.782222,91.549548,6181975,0.961070,1.071835
7,Estudios de Posgrado,72.042038,64.857167,79.226908,542916,3.665818,5.088443


Estas estimaciones se pueden interpretar de la siguiente manera:
 > Del 100% de personas con ________ , el ____% no estudia

Por ejemplo, del 100% de personas con "secundaria completa", el 88.14% no estudia.

### 4.5 ¿Con qué proporción se mencionaron ciertas respuesta de la pregunta seleccionada? 
### Desagregado por sexo y grupo etario
Una de las desagregaciones más solicitadas es por sexo y grupo etario, por ello se creó la función `tabulados_prevalencias`.

Los argumentos de esta función son los mismos que en la función `prevalencias_des`, con excepción de `var_sexo` y `var_ge` para indicar los nombres de las variables que contienen la información del sexo y el grupo etario en la encuesta.

Ahora bien, para usar esta función, vamos a crear una variable correspondiente a los grupos etarios de nuestra encuesta y estructuraremos la variable de género que se tiene:

In [26]:
# ---------- # Adecuaciones a las bases de datos # ------------------ #

# Creación de grupos etarios 
#(los grupos etarios se pueden cambiar si así se desea) 
grupos_etarios=['12-17', '18-34', '35-59', '60-75']
divisiones=[int(x[:2])-1 for x in grupos_etarios]+[int(grupos_etarios[-1][-2:])]
b.insert(len(b.columns),'grupo_etario',pd.cut(list(b['ds3']), bins=divisiones, labels=grupos_etarios))

# Cambio de nombre y valores de las columnas entidad y sexo
b.rename(columns={'entidad':'cve_ent','desc_ent':'nom_ent','ds2':'sexo'}, inplace=True)
valores_sexo = {1:'Hombre', 2:'Mujer'}
b['sexo'] = b['sexo'].map(valores_sexo).fillna(b['sexo'])
b['nom_ent']=b['nom_ent'].str.strip().apply(arreglar_palabras)

Con estas adecuaciones es posible utilizar `tabulados_prevalencias` para estimar la proporción de personas en cada nivel escolar que no asisten a la escuela desagregado por género y grupo etario:

In [28]:
tabulados_prevalencias(b = b,
                       lista_vars = ['ds8'],
                       dicc={1:1, 2:1, 3:0},
                       var_des = ['ds9'],
                       alp = 0.95,
                       var_sexo='sexo',
                       var_ge='grupo_etario',
                       ponderador = 'factor_exp',
                       estrato= 'estrato',
                       upm = 'upm')

,sexo,grupo_etario,ds9,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
9,Mujeres y hombres,Población total,0,100.000000,100.000000,100.000000,1905531,0.000000,0.000000
3,Mujeres y hombres,Población total,1,90.387407,88.842200,91.932614,5811612,0.788385,0.872229
5,Mujeres y hombres,Población total,2,82.431266,81.057882,83.804650,10671142,0.700719,0.850065
4,Mujeres y hombres,Población total,3,47.288119,45.279154,49.297085,4066190,1.025001,2.167566
0,Mujeres y hombres,Población total,4,88.140337,87.301344,88.979330,18880798,0.428066,0.485664
...,...,...,...,...,...,...,...,...,...
81,Hombre,60-75,99,95.948251,86.146261,105.750242,6749,5.001107,5.212297
78,Mujer,12-17,99,37.823834,0.000000,82.008822,1460,22.543775,59.602036
76,Mujer,18-34,99,100.000000,100.000000,100.000000,4347,0.000000,0.000000
74,Mujer,35-59,99,97.911906,94.702574,101.121238,43702,1.637444,1.672365
